## imports

In [1]:
import os
import numpy as np
from shapely import geometry
from osgeo import ogr

from utils.print_utils import printProgress, printSuccess, printWarning
        
# directory where the data will be accessed and stored
data_partition = 'c:\\data\\'
country = 'cco'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')
results_dir_path = os.path.join(base_dir_path, 'regions')
sites = os.listdir(sites_dir_path)

## functions

In [2]:
def polygon_from_kml(fn):

    # read .kml file
    with open(fn) as kmlFile:
        doc = kmlFile.read()
        # parse to find coordinates field
    str1 = '<coordinates>'
    str2 = '</coordinates>'
    subdoc = doc[doc.find(str1) + len(str1):doc.find(str2)]
    
    coordlist = subdoc.split(',')

    polygon = []
    for coord_pair in range(int(len(coordlist) / 2)):
        lat = float(coordlist[coord_pair * 2].replace('0 ', ''))
        lon = float(coordlist[coord_pair * 2 + 1].replace('0 ', ''))
        polygon.append([lat, lon])

    return polygon


def smallest_rectangle(polygon):
    
    multipoints = geometry.Polygon(polygon)
    polygon_geom = multipoints.envelope
    coords_polygon = np.array(polygon_geom.exterior.coords)
    polygon_rect = [[_[0], _[1]] for _ in coords_polygon]
    
    return polygon_rect


def poly2wkt(polygon):
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    
    for pt in polygon:
        ring.AddPoint(pt[0], pt[1])

    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)

    print(poly.ExportToWkt())
#    print (poly.ExportToJson())
#    polygon_str = str(poly.ExportToWkt())
#    print(f'{polygon_str:.2f}')
    
def wkt2shp(wkt_list, shp_file):

    # Define a linestring feature geometry with one attribute
    schema = {
        'geometry': 'LineString',
        'properties': {'id': 'int'},
    }

    # Write a new shapefile
    with fiona.open(shp_file, 'w', 'ESRI Shapefile', schema) as c:
        for i, line in enumerate(wkt_list):
            shape = shapely.wkt.loads(line)
            c.write({
                    'geometry': mapping(shape),
                    'properties': {'id': i},
                    })

    return

## run the code

In [3]:
for site in sites:
    
    site_name = site[:site.find('.')]
    
    printProgress(f'processing: {site_name}')
    print()
        
    kml_filepath = os.path.join(sites_dir_path, site)
    print(kml_filepath)
    kml_polygon = polygon_from_kml(kml_filepath)
    
    roi_polygon = smallest_rectangle(kml_polygon)

    poly2wkt(roi_polygon)
            
print()
printProgress('FINISHED ...')

progress  >>>  processing: Portdinllaen

c:\data\cco\sites\Portdinllaen.kml
POLYGON ((-4.58849986226323 52.9293354441919 0,-4.4941225140105 52.9293354441919 0,-4.4941225140105 52.9582823481204 0,-4.58849986226323 52.9582823481204 0,-4.58849986226323 52.9293354441919 0))

progress  >>>  FINISHED ...
